In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
# Data collection and cleaning/prep

In [4]:
%matplotlib inline
import sys
from pathlib import Path

import pandas as pd

import spacy
from spacy import displacy
from textacy.extract import ngrams, entities

import numpy as np
import re
import pandas as pd


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# spacy, textblob and nltk for language processing
from textblob import TextBlob, Word
import nltk
from nltk.stem.snowball import SnowballStemmer

# sklearn for feature extraction & modeling
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [5]:


nltk.download('wordnet')

nlp = spacy.load('en_core_web_lg') 

DATA_DIR = Path('data')

Label_movie_data = pd.read_csv('movie_data.csv')

nltk.download('punkt')

nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JBarr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JBarr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\JBarr\AppData\Roaming\nltk_data...


True

In [6]:

Unlabel_earningsWords = pd.read_csv('earningsWords.csv')

Unlabel_earningsWords = Unlabel_earningsWords[['Ticker','article']]

Group_earningsWords = Unlabel_earningsWords.groupby(['Ticker', 'article']).max()


# ---->  works 

Zxearnings = str(Group_earningsWords.loc['Zx'].index)



filesE = (DATA_DIR / 'earnings_calls').glob('**/*.csv')
comp_eranings = ''
for i, fileE in enumerate(filesE):
    with fileE.open(encoding='latin1') as f:
        lines = f.readlines()
        body = ' '.join([l.strip() for l in lines[1:]]).strip()
        comp_eranings += body
        
        
        
earnsText = comp_eranings[:55000]     


In [7]:


files = (DATA_DIR / 'bbc').glob('**/*.txt')
bbc_articles = []
for i, file in enumerate(sorted(list(files))):
    with file.open(encoding='latin1') as f:
        lines = f.readlines()
        body = ' '.join([l.strip() for l in lines[1:]]).strip()
        bbc_articles.append(body)
        
        
        
path = Path('data', 'bbc')
files = sorted(list(path.glob('**/*.txt')))
doc_list = []
for i, file in enumerate(files):
    topic = file.parts[-2]
    article = file.read_text(encoding='latin1').split('\n')
    heading = article[0].strip()
    body = ' '.join([l.strip() for l in article[1:]]).strip()
    doc_list.append([topic, heading, body])
    


docs = pd.DataFrame(doc_list, columns=['topic', 'heading', 'body'])
docs.info()            

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   topic    2225 non-null   object
 1   heading  2225 non-null   object
 2   body     2225 non-null   object
dtypes: object(3)
memory usage: 52.3+ KB


In [8]:
article = docs.sample(1).squeeze()


parsed_body = TextBlob(article.body)



display(print(f'Topic:\t{article.topic.capitalize()}\n\n{article.heading}\n'),
print(article.body.strip()),

    parsed_body.words, 
        parsed_body.sentences)

Topic:	Business

German economy rebounds

Germany's economy, the biggest among the 12 countries sharing the euro, grew at its fastest rate in four years during 2004, driven by strong exports.  Gross domestic product (GDP) rose by 1.7% last year, the statistical office said. The economy contracted in 2003. Foreign sales increased by 8.2% last year, compared with a 0.3% slide in private consumption. Concerns remain, however, over the strength of the euro, weak domestic demand and a sluggish labour market. The European Central Bank (ECB) left its benchmark interest rate unchanged at 2% on Thursday. It is the nineteenth month in a row that the ECB has not moved borrowing costs. Economists predict that an increase is unlikely to come until the second half of 2005, with growth set to sputter rather than ignite.  "During 2004 we profited from the fact that the world economy was strong," said Stefan Schilbe, analyst at HSBC Trinkaus & Burkhardt. "If exports weaken and domestic growth remains p

None

None

WordList(['Germany', "'s", 'economy', 'the', 'biggest', 'among', 'the', '12', 'countries', 'sharing', 'the', 'euro', 'grew', 'at', 'its', 'fastest', 'rate', 'in', 'four', 'years', 'during', '2004', 'driven', 'by', 'strong', 'exports', 'Gross', 'domestic', 'product', 'GDP', 'rose', 'by', '1.7', 'last', 'year', 'the', 'statistical', 'office', 'said', 'The', 'economy', 'contracted', 'in', '2003', 'Foreign', 'sales', 'increased', 'by', '8.2', 'last', 'year', 'compared', 'with', 'a', '0.3', 'slide', 'in', 'private', 'consumption', 'Concerns', 'remain', 'however', 'over', 'the', 'strength', 'of', 'the', 'euro', 'weak', 'domestic', 'demand', 'and', 'a', 'sluggish', 'labour', 'market', 'The', 'European', 'Central', 'Bank', 'ECB', 'left', 'its', 'benchmark', 'interest', 'rate', 'unchanged', 'at', '2', 'on', 'Thursday', 'It', 'is', 'the', 'nineteenth', 'month', 'in', 'a', 'row', 'that', 'the', 'ECB', 'has', 'not', 'moved', 'borrowing', 'costs', 'Economists', 'predict', 'that', 'an', 'increase', 

[Sentence("Germany's economy, the biggest among the 12 countries sharing the euro, grew at its fastest rate in four years during 2004, driven by strong exports."),
 Sentence("Gross domestic product (GDP) rose by 1.7% last year, the statistical office said."),
 Sentence("The economy contracted in 2003."),
 Sentence("Foreign sales increased by 8.2% last year, compared with a 0.3% slide in private consumption."),
 Sentence("Concerns remain, however, over the strength of the euro, weak domestic demand and a sluggish labour market."),
 Sentence("The European Central Bank (ECB) left its benchmark interest rate unchanged at 2% on Thursday."),
 Sentence("It is the nineteenth month in a row that the ECB has not moved borrowing costs."),
 Sentence("Economists predict that an increase is unlikely to come until the second half of 2005, with growth set to sputter rather than ignite."),
 Sentence(""During 2004 we profited from the fact that the world economy was strong," said Stefan Schilbe, analyst

In [9]:
# Initialize stemmer.
stemmer = SnowballStemmer('english')

# Stem each word.
[(word, stemmer.stem(word)) for i, word in enumerate(parsed_body.words) 
 if word.lower() != stemmer.stem(parsed_body.words[i])]

[('Germany', 'germani'),
 ('economy', 'economi'),
 ('countries', 'countri'),
 ('sharing', 'share'),
 ('its', 'it'),
 ('years', 'year'),
 ('during', 'dure'),
 ('exports', 'export'),
 ('domestic', 'domest'),
 ('statistical', 'statist'),
 ('office', 'offic'),
 ('economy', 'economi'),
 ('contracted', 'contract'),
 ('sales', 'sale'),
 ('increased', 'increas'),
 ('compared', 'compar'),
 ('private', 'privat'),
 ('consumption', 'consumpt'),
 ('Concerns', 'concern'),
 ('however', 'howev'),
 ('domestic', 'domest'),
 ('its', 'it'),
 ('unchanged', 'unchang'),
 ('moved', 'move'),
 ('borrowing', 'borrow'),
 ('costs', 'cost'),
 ('Economists', 'economist'),
 ('increase', 'increas'),
 ('unlikely', 'unlik'),
 ('ignite', 'ignit'),
 ('During', 'dure'),
 ('profited', 'profit'),
 ('economy', 'economi'),
 ('Schilbe', 'schilb'),
 ('exports', 'export'),
 ('domestic', 'domest'),
 ('remains', 'remain'),
 ('Many', 'mani'),
 ('consumers', 'consum'),
 ('spooked', 'spook'),
 ('unsettled', 'unsettl'),
 ('government',

In [10]:
[(word, word.lemmatize()) for i, word in enumerate(parsed_body.words) 
 if word != parsed_body.words[i].lemmatize()]


[('countries', 'country'),
 ('its', 'it'),
 ('years', 'year'),
 ('exports', 'export'),
 ('sales', 'sale'),
 ('its', 'it'),
 ('has', 'ha'),
 ('costs', 'cost'),
 ('was', 'wa'),
 ('exports', 'export'),
 ('consumers', 'consumer'),
 ('attempts', 'attempt'),
 ('companies', 'company'),
 ('talks', 'talk'),
 ('unions', 'union'),
 ('jobs', 'job'),
 ('costs', 'cost'),
 ('measures', 'measure')]

In [15]:
[(word, word.lemmatize(pos='v')) for i, word in enumerate(parsed_body.words) 
 if word != parsed_body.words[i].lemmatize(pos='v')]

[('sharing', 'share'),
 ('grew', 'grow'),
 ('driven', 'drive'),
 ('exports', 'export'),
 ('rose', 'rise'),
 ('said', 'say'),
 ('contracted', 'contract'),
 ('increased', 'increase'),
 ('compared', 'compare'),
 ('left', 'leave'),
 ('is', 'be'),
 ('has', 'have'),
 ('moved', 'move'),
 ('borrowing', 'borrow'),
 ('costs', 'cost'),
 ('is', 'be'),
 ('profited', 'profit'),
 ('was', 'be'),
 ('said', 'say'),
 ('exports', 'export'),
 ('remains', 'remain'),
 ('been', 'be'),
 ('spooked', 'spook'),
 ('unsettled', 'unsettle'),
 ('attempts', 'attempt'),
 ('companies', 'company'),
 ('including', 'include'),
 ('spent', 'spend'),
 ('talks', 'talk'),
 ('trimming', 'trim'),
 ('jobs', 'job'),
 ('costs', 'cost'),
 ('warned', 'warn'),
 ('are', 'be'),
 ('cutting', 'cut'),
 ('measures', 'measure')]

In [16]:
parsed_body.sentiment

Sentiment(polarity=0.009434865900383142, subjectivity=0.31810344827586207)

In [18]:
parsed_body.sentiment_assessments


Sentiment(polarity=0.009434865900383142, subjectivity=0.31810344827586207, assessments=[(['strong'], 0.4333333333333333, 0.7333333333333333, None), (['gross'], 0.0, 0.0, None), (['domestic'], 0.0, 0.1, None), (['rose'], 0.6, 0.95, None), (['last'], 0.0, 0.06666666666666667, None), (['foreign'], -0.125, 0.125, None), (['last'], 0.0, 0.06666666666666667, None), (['private'], 0.0, 0.375, None), (['weak'], -0.375, 0.625, None), (['domestic'], 0.0, 0.1, None), (['european'], 0.0, 0.0, None), (['central'], 0.0, 0.25, None), (['left'], 0.0, 0.0, None), (['unlikely'], -0.5, 0.5, None), (['second'], 0.0, 0.0, None), (['half'], -0.16666666666666666, 0.16666666666666666, None), (['strong'], 0.4333333333333333, 0.7333333333333333, None), (['domestic'], 0.0, 0.1, None), (['poor'], -0.4, 0.6, None), (['much'], 0.2, 0.2, None), (['many'], 0.5, 0.5, None), (['german'], 0.0, 0.0, None), (['corporate'], 0.0, 0.0, None), (['major'], 0.0625, 0.5, None), (['spent'], -0.1, 0.1, None), (['much'], 0.2, 0.2, N

In [19]:
def lemmatizer(text):
    words = TextBlob(text.lower()).words
    return [word.lemmatize() for word in words]


vectorizer = CountVectorizer(analyzer=lemmatizer, decode_error='replace')

In [11]:
def get_attributes(f):
    print([a for a in dir(f) if not a.startswith('_')], end=' ')

In [12]:
doc = nlp(bbc_articles[0])


sentences = [s for s in doc.sents]
sentences[:3]

[Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.  ,
 The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales.,
 TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.]

In [13]:
docTwo = nlp(Zxearnings)
sentencesTwo = [s for s in docTwo.sents]
tunkofearnings = sentencesTwo[24:110]

In [14]:

pd.Series([token.text for token in tunkofearnings])

0     So, let me start with the brief discussion on ...
1     For the third quarter of 2018 China's GDP grow...
2     This slower GDP growth was the weakest growth ...
3     Industrial production in the month of Septembe...
4     According the data reported by the China Assoc...
                            ...                        
81    Basic and diluted earnings per ordinary share ...
82    Now for our balance sheet as of September 30, ...
83    Pledged deposits were RMB37.7 million or US$5....
84    For the bank borrowings were RMB558 million or...
85    Total equity attributable to owners of the com...
Length: 86, dtype: object

In [283]:
pd.Series([n.text for n in ngrams(docTwo, n=2, min_freq=2)]).value_counts()

Zenix Auto            10
looking statements     7
September 30           7
aluminum wheel         7
unit sales             7
                      ..
Steel Wheel            1
Net loss               1
commercial vehicle     1
Tubed steel            1
total unit             1
Length: 142, dtype: int64